In [3]:
import re
from collections import Counter
import tensorflow as tf
import os

def read_lyrics():
    path_to_file = 'data/eminem.txt'
    lyrics = open(path_to_file, 'rb').read().decode(encoding='utf-8')

    return lyrics

text = read_lyrics()
print(f'Length of text: {len(text)} characters')


Length of text: 167200 characters


In [4]:
vocab = sorted(set(text))

chars = tf.strings.unicode_split(text, input_encoding='UTF-8')

ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

ids = ids_from_chars(chars)

ids_dataset = tf.data.Dataset.from_tensor_slices(ids)

seq_length = 100

sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


2023-03-07 11:55:47.771009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [6]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [7]:
class RNNModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [8]:
model = RNNModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [9]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 100) # (batch_size, sequence_length, vocab_size)


In [10]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [11]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback], use_multiprocessing=True)


Epoch 1/10
25/25 [==============================] - 60s 2s/step - loss: 3.9428
Epoch 2/10
25/25 [==============================] - 57s 2s/step - loss: 3.0606
Epoch 3/10
25/25 [==============================] - 62s 2s/step - loss: 2.7056
Epoch 4/10
25/25 [==============================] - 63s 2s/step - loss: 2.4733
Epoch 5/10
25/25 [==============================] - 67s 3s/step - loss: 2.3625
Epoch 6/10
 1/25 [>.............................] - ETA: 1:10 - loss: 2.3371

KeyboardInterrupt: 

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [ ]:
import time

start = time.time()
states = None
result = []

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

signtosnlnkscmntkanmproo-dalrmosnindntcaontsldas1asganimltadltoaasaagomdoalkrnacilnmtm-idkvrjalr-oijmroramsilodrrnobntposcsrtaloogs-ttarjn-bnrgstanwgtingclmoιmirtdintmkt-wnitaaodoooddwiltmomamkkcoropoocgiinnkanmwsnmmionnmoaoiiodtolstsaltt-nolroooobdaicow-gainnsipaaaat-iaiasovrsanasttcoibpnrgcnakrmaamsd-dbob-otgltaaooootstbmrmknltnipr-ipcoaglnmaomadsiiootdtmoatroctotbbtadnttaaobbtgnlvoopaavgrli"rvaosnmbtinrtatnndnnkoklvli-nmnoasaniaonniackaloiasrookooc-ootdtiiwnico-oolosasmraoortokavanaoawndmanaotcdtatoanmoloinotnnatsmlpiwwntaoriwpagnigtttoswgjnis-ooainb"-otntnoncsoi-tooroasnlcnmcwarltpbmrtmrtrwimiwwi-soslitoiatoilnwanwniwslilotolatnorinraionoitpaoabrwnvtoncscnklnmmclkoattnmmnoo-dtrosl-otoiwonodo-ikapitraottlglroittsaawmotodntntdoipttaonanwaokcaoanonpmbwotoatonkolg--nkioltivmtntasiltddalt"sdatstnoosccmotopoasmitnrokasattalmnaldsmkogtlosdapisnctbavni-ottikwonktniadlraitabtmngionmottosalioswaoamgkalorlnolrrsoorsrloocoomcrtangoowbbmpocanbokn-igaaonadisaosmjolroogdoragamngootonsn"mairndlbsn